# Polymarket Integration - WebSocket, Markets & Arbitrage Detection

**Prerequisites:** Run `02_complete_testing.ipynb` first!

**What we'll test:**
1. ✅ Generate Polymarket API credentials
2. ✅ Initialize Polymarket client
3. ✅ Fetch available markets
4. ✅ Get order book data
5. ✅ Calculate arbitrage opportunities
6. ✅ Test fee calculations

**Run each cell sequentially!**

---
## Section 1: Setup & Imports

In [ ]:
print("🔍 Starting Polymarket Integration Testing...")
print("="*60)
print("")

print("📦 Importing libraries...")
import sys
import os
import asyncio
import time
from datetime import datetime
from pathlib import Path
from dotenv import load_dotenv

# Web3 & Blockchain
from web3 import Web3

# Polymarket SDK
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import ApiCreds
from py_clob_client.constants import POLYGON

# Speed optimizations
import orjson

# Data analysis
import pandas as pd
import numpy as np

print("✅ All imports successful!")
print("")
print("✅ Section 1 complete!")

---
## Section 2: Load Configuration

In [ ]:
print("🔍 Loading configuration...")
print("="*60)
print("")

# Load .env file
project_root = Path().absolute().parent
env_path = project_root / '.env'
load_dotenv(env_path)

# Get config
POLYGON_RPC_URL = os.getenv('POLYGON_RPC_URL')
WALLET_ADDRESS = os.getenv('WALLET_ADDRESS')
WALLET_PRIVATE_KEY = os.getenv('WALLET_PRIVATE_KEY')
DRY_RUN_MODE = os.getenv('DRY_RUN_MODE', 'true').lower() == 'true'
POLYMARKET_HOST = os.getenv('POLYMARKET_HOST', 'https://clob.polymarket.com')

print("📊 Configuration:")
print(f"   💰 Wallet: {WALLET_ADDRESS}")
print(f"   🧪 Dry Run: {DRY_RUN_MODE}")
print(f"   🌐 Polymarket Host: {POLYMARKET_HOST}")
print("")
print("✅ Section 2 complete!")

---
## Section 3: Generate Polymarket API Credentials

**IMPORTANT:** This creates API keys for Polymarket CLOB.
- Keys are deterministically derived from your wallet's private key
- Same wallet = same API keys every time
- No need to store separately (can regenerate anytime)

In [ ]:
print("🔍 Generating Polymarket API credentials...")
print("="*60)
print("")

# Check if we have private key
if not WALLET_PRIVATE_KEY or WALLET_PRIVATE_KEY == '0xYourPrivateKeyHere':
    print("❌ ERROR: WALLET_PRIVATE_KEY not set in .env file!")
    print("")
    print("⚠️ You need your private key to generate API credentials.")
    print("")
    print("📋 Options:")
    print("   1. Add your private key to .env file (WALLET_PRIVATE_KEY)")
    print("   2. Export from MetaMask: Account Details > Export Private Key")
    print("   3. NEVER share your private key with anyone!")
    print("")
    print("⚠️ Skipping credential generation for now...")
    api_creds_ready = False
else:
    try:
        print("⏱️ Validating private key format...")
        
        # Validate private key format
        if not WALLET_PRIVATE_KEY.startswith('0x'):
            print("❌ ERROR: Private key must start with '0x'")
            print("   Example: 0x1234567890abcdef...")
            api_creds_ready = False
        elif len(WALLET_PRIVATE_KEY) != 66:  # 0x + 64 hex chars
            print(f"⚠️ WARNING: Private key length is {len(WALLET_PRIVATE_KEY)}, expected 66")
            print("   Should be: 0x + 64 hexadecimal characters")
            api_creds_ready = False
        else:
            print("✅ Private key format valid!")
            print("")
            print("💡 Note: ClobClient will auto-generate API credentials")
            print("   from your private key in the next section")
            print("")
            print("🔑 Credentials are deterministically derived:")
            print("   - Same wallet = same API keys every time")
            print("   - No need to store separately")
            print("   - Handled automatically by py-clob-client")
            api_creds_ready = True
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())
        api_creds_ready = False

print("")
print("✅ Section 3 complete!")

---
## Section 4: Initialize Polymarket Client

In [ ]:
print("🔍 Initializing Polymarket CLOB client...")
print("="*60)
print("")

if not api_creds_ready:
    print("❌ Cannot initialize client - no valid private key")
    print("   Add your private key to .env and run Section 3 again")
    client = None
else:
    try:
        print("⏱️ Creating ClobClient with private key...")
        
        # Initialize client directly with private key
        # The library will auto-generate API credentials internally
        client = ClobClient(
            host=POLYMARKET_HOST,
            key=WALLET_PRIVATE_KEY,  # Private key, not API key
            chain_id=POLYGON
        )
        
        print("✅ Client initialized!")
        print("")
        print("📊 Client info:")
        print(f"   🌐 Host: {POLYMARKET_HOST}")
        print(f"   🔗 Chain ID: {POLYGON}")
        print(f"   💰 Address: {WALLET_ADDRESS}")
        print("")
        print("💡 API credentials auto-generated by library")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        print("")
        import traceback
        print(traceback.format_exc())
        client = None

print("")
print("✅ Section 4 complete!")

---
## Section 5: Fetch Available Markets

Get a list of active markets on Polymarket

In [ ]:
print("🔍 Fetching available markets...")
print("="*60)
print("")

if client is None:
    print("❌ No client - skipping market fetch")
    markets = []
else:
    try:
        print("⏱️ Fetching markets from Polymarket CLOB...")
        
        # Get markets
        markets_response = client.get_markets()
        
        # Convert to list if needed
        if isinstance(markets_response, dict):
            markets = markets_response.get('data', [])
        else:
            markets = markets_response
        
        print(f"✅ Found {len(markets)} total markets")
        print("")
        
        # Filter for CURRENT, OPEN markets
        if len(markets) > 0:
            df = pd.DataFrame(markets)
            
            print("📊 Filtering for current, open markets...")
            print("")
            
            # IMPORTANT: Filter for markets that are actually open NOW
            # closed=False is the key - we want markets still trading
            df_open = df[df['closed'] == False]
            
            print(f"   ✅ {len(df_open)} markets NOT closed")
            
            # Check accepting_orders
            if len(df_open) > 0 and 'accepting_orders' in df_open.columns:
                df_accepting = df_open[df_open['accepting_orders'] == True]
                print(f"   📊 {len(df_accepting)} of those accepting orders")
            
            print("")
            
            # Use open markets even if not accepting orders
            if len(df_open) > 0:
                print(f"📊 Showing {len(df_open)} OPEN markets (closed=False):")
                print("")
                
                for idx, row in df_open.head(10).iterrows():
                    question = row.get('question', 'N/A')
                    end_date = row.get('end_date_iso', 'N/A')
                    accepting = row.get('accepting_orders', 'N/A')
                    print(f"   📈 {question[:65]}")
                    print(f"      📅 Ends: {end_date}")
                    print(f"      📊 Accepting Orders: {accepting}")
                    print(f"      🆔 ID: {row.get('condition_id', 'N/A')[:20]}...")
                    print("")
                
                # Update markets list to only include open markets
                markets = df_open.to_dict('records')
            else:
                print("   ❌ No open markets found (all closed=True)!")
                markets = []
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())
        markets = []

print("")
print("✅ Section 5 complete!")

---
## Section 6: Select a Market to Monitor

Choose a high-volume market for testing

In [ ]:
print("🔍 Selecting a market to monitor...")
print("="*60)
print("")

if len(markets) == 0:
    print("❌ No markets available")
    selected_market = None
    token_ids = {}
else:
    try:
        # Convert to DataFrame
        df = pd.DataFrame(markets)
        
        # We already have filtered markets (closed=False)
        # Just take the first one
        selected_market = df.iloc[0].to_dict()
        
        # Extract token IDs (each market has YES and NO tokens)
        tokens = selected_market.get('tokens', [])
        token_ids = {}
        
        if tokens and len(tokens) >= 2:
            # Usually tokens[0] is YES, tokens[1] is NO
            token_ids = {
                'yes': tokens[0].get('token_id'),
                'no': tokens[1].get('token_id')
            }
        
        print("✅ Selected market:")
        print("")
        print(f"📊 Market Details:")
        print(f"   ❓ Question: {selected_market.get('question', 'N/A')}")
        print(f"   🆔 Condition ID: {selected_market.get('condition_id', 'N/A')}")
        print(f"   📅 End Date: {selected_market.get('end_date_iso', 'N/A')}")
        print(f"   ✅ Active: {selected_market.get('active', 'N/A')}")
        print(f"   🔒 Closed: {selected_market.get('closed', 'N/A')}")
        print(f"   📊 Accepting Orders: {selected_market.get('accepting_orders', 'N/A')}")
        print("")
        
        if token_ids and 'yes' in token_ids and 'no' in token_ids:
            print(f"🎯 Token IDs:")
            print(f"   🟢 YES Token: {token_ids.get('yes', 'N/A')}")
            print(f"   🔴 NO Token: {token_ids.get('no', 'N/A')}")
        else:
            print(f"⚠️ Could not extract token IDs")
            print(f"   Tokens data: {tokens}")
        print("")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())
        selected_market = None
        token_ids = {}

print("")
print("✅ Section 6 complete!")

---
## Section 7: Fetch Order Book Data

Get current order books for YES and NO tokens

In [ ]:
print("🔍 Fetching order book data...")
print("="*60)
print("")

if client is None or not token_ids or 'yes' not in token_ids:
    print("❌ Cannot fetch order books - no client or token IDs")
    orderbook_data = None
else:
    try:
        print("⏱️ Fetching order books from CLOB...")
        
        # Get order books for each token separately
        yes_token_id = token_ids['yes']
        no_token_id = token_ids['no']
        
        print(f"   🟢 Fetching YES token: {yes_token_id[:20]}...")
        yes_orderbook = client.get_order_book(yes_token_id)
        
        print(f"   🔴 Fetching NO token: {no_token_id[:20]}...")
        no_orderbook = client.get_order_book(no_token_id)
        
        # Combine into one structure
        orderbook_data = {
            'yes': yes_orderbook,
            'no': no_orderbook
        }
        
        print("✅ Order book data retrieved!")
        print("")
        
        # Display order books
        print("📊 Order Book Summary:")
        print("")
        
        # YES token
        print("   💹 YES Token:")
        if 'asks' in yes_orderbook and len(yes_orderbook['asks']) > 0:
            best_ask = yes_orderbook['asks'][0]
            print(f"      🟢 Best Ask (Buy): ${best_ask['price']} (Size: {best_ask['size']})")
        else:
            print(f"      ⚠️ No asks available")
            
        if 'bids' in yes_orderbook and len(yes_orderbook['bids']) > 0:
            best_bid = yes_orderbook['bids'][0]
            print(f"      🔴 Best Bid (Sell): ${best_bid['price']} (Size: {best_bid['size']})")
        else:
            print(f"      ⚠️ No bids available")
        print("")
        
        # NO token
        print("   💹 NO Token:")
        if 'asks' in no_orderbook and len(no_orderbook['asks']) > 0:
            best_ask = no_orderbook['asks'][0]
            print(f"      🟢 Best Ask (Buy): ${best_ask['price']} (Size: {best_ask['size']})")
        else:
            print(f"      ⚠️ No asks available")
            
        if 'bids' in no_orderbook and len(no_orderbook['bids']) > 0:
            best_bid = no_orderbook['bids'][0]
            print(f"      🔴 Best Bid (Sell): ${best_bid['price']} (Size: {best_bid['size']})")
        else:
            print(f"      ⚠️ No bids available")
        print("")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())
        orderbook_data = None

print("")
print("✅ Section 7 complete!")

---
## Section 8: Calculate Arbitrage Opportunity

Check if buying both YES and NO tokens costs less than $1.00

In [ ]:
print("🔍 Calculating arbitrage opportunity...")
print("="*60)
print("")

if orderbook_data is None:
    print("❌ No order book data available")
else:
    try:
        print("⏱️ Analyzing order books...")
        print("")
        
        # Extract prices
        yes_ask = None
        no_ask = None
        
        # Get YES token best ask (price to BUY YES)
        if 'yes' in orderbook_data:
            if 'asks' in orderbook_data['yes'] and len(orderbook_data['yes']['asks']) > 0:
                yes_ask = float(orderbook_data['yes']['asks'][0]['price'])
        
        # Get NO token best ask (price to BUY NO)
        if 'no' in orderbook_data:
            if 'asks' in orderbook_data['no'] and len(orderbook_data['no']['asks']) > 0:
                no_ask = float(orderbook_data['no']['asks'][0]['price'])
        
        if yes_ask is not None and no_ask is not None:
            print("📊 Price Analysis:")
            print(f"   🟢 YES Token Ask: ${yes_ask:.4f}")
            print(f"   🔴 NO Token Ask: ${no_ask:.4f}")
            print("")
            
            # Calculate total cost
            total_cost = yes_ask + no_ask
            print(f"   💰 Total Cost (YES + NO): ${total_cost:.4f}")
            print("")
            
            # Calculate gross edge (before fees)
            gross_edge = 1.0 - total_cost
            gross_edge_bps = gross_edge * 10000
            
            print(f"   📈 Gross Edge: ${gross_edge:.4f} ({gross_edge_bps:.1f} bps)")
            print("")
            
            # Estimate fees (simplified)
            # Polymarket fees vary by market, but typically:
            # - Maker fee: 0% to 0.1%
            # - Taker fee: 0.1% to 0.2%
            # - Gas cost: ~$0.02-0.05 per merge
            
            trade_size = 100.0  # Assume $100 trade
            taker_fee_rate = 0.002  # 0.2% worst case
            total_fees = trade_size * taker_fee_rate * 2  # Both YES and NO orders
            gas_cost = 0.05
            slippage_buffer = trade_size * 0.002  # 0.2%
            
            total_costs = total_fees + gas_cost + slippage_buffer
            costs_per_dollar = total_costs / trade_size
            
            print("   💸 Estimated Costs (per $100 trade):")
            print(f"      - Trading fees: ${total_fees:.2f}")
            print(f"      - Gas cost: ${gas_cost:.2f}")
            print(f"      - Slippage buffer: ${slippage_buffer:.2f}")
            print(f"      - Total: ${total_costs:.2f} ({costs_per_dollar*100:.2f}%)")
            print("")
            
            # Calculate net edge
            net_edge = gross_edge - costs_per_dollar
            net_edge_bps = net_edge * 10000
            
            print(f"   💹 Net Edge (after costs): ${net_edge:.4f} ({net_edge_bps:.1f} bps)")
            print("")
            
            # Profitability check
            if net_edge > 0.01:  # 1% profit threshold
                print("   ✅ PROFITABLE ARBITRAGE DETECTED!")
                print(f"   💰 Estimated profit: ${net_edge * trade_size:.2f} per $100 trade")
            elif net_edge > 0:
                print("   ⚠️ Marginal opportunity (< 1% profit)")
                print(f"   💰 Estimated profit: ${net_edge * trade_size:.2f} per $100 trade")
            else:
                print("   ❌ NOT PROFITABLE (costs exceed edge)")
                print(f"   💸 Would lose: ${abs(net_edge * trade_size):.2f} per $100 trade")
        else:
            print("❌ Could not extract prices from order book")
            print(f"   YES ask: {yes_ask}")
            print(f"   NO ask: {no_ask}")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        print(traceback.format_exc())

print("")
print("✅ Section 8 complete!")

---
## Section 9: Fee Calculation Function

Create a reusable function to calculate required edge for profitability

In [ ]:
print("🔍 Creating fee calculation function...")
print("="*60)
print("")

def calculate_min_edge(trade_size_usd, maker_fee=0.001, taker_fee=0.002):
    """
    Calculate minimum edge needed for profitability.
    
    Args:
        trade_size_usd: Dollar amount of trade
        maker_fee: Maker fee rate (default 0.1%)
        taker_fee: Taker fee rate (default 0.2%)
    
    Returns:
        Dictionary with cost breakdown and minimum edge
    """
    print(f"   🔍 Calculating for ${trade_size_usd} trade...")
    
    # Trading fees (worst case: both orders are taker)
    clob_fees = trade_size_usd * taker_fee * 2  # Both YES and NO orders
    
    # Gas cost (relatively fixed on Polygon)
    gas_cost = 0.05
    
    # Slippage (scales with trade size)
    slippage = trade_size_usd * 0.002  # 0.2%
    
    # Total costs
    total = clob_fees + gas_cost + slippage
    
    # As percentage of trade
    min_edge_pct = total / trade_size_usd
    
    # Add 30% safety buffer
    min_edge_safe = min_edge_pct * 1.3
    
    return {
        'trade_size': trade_size_usd,
        'clob_fees': clob_fees,
        'gas_cost': gas_cost,
        'slippage': slippage,
        'total_cost': total,
        'min_edge_pct': min_edge_pct,
        'min_edge_safe': min_edge_safe,
        'min_edge_bps': min_edge_pct * 10000,
        'min_edge_safe_bps': min_edge_safe * 10000
    }

print("✅ Function created!")
print("")

# Test with different trade sizes
print("📊 Testing with different trade sizes:")
print("")

for trade_size in [10, 50, 100, 200]:
    result = calculate_min_edge(trade_size)
    print(f"   💰 ${trade_size} trade:")
    print(f"      Total costs: ${result['total_cost']:.2f}")
    print(f"      Min edge: {result['min_edge_pct']*100:.2f}% ({result['min_edge_bps']:.1f} bps)")
    print(f"      Safe edge: {result['min_edge_safe']*100:.2f}% ({result['min_edge_safe_bps']:.1f} bps)")
    print("")

print("")
print("✅ Section 9 complete!")

---
## Section 10: Final Summary

In [ ]:
print("="*60)
print("🎉 POLYMARKET INTEGRATION TEST COMPLETE!")
print("="*60)
print("")
print("✅ Section 1: Setup & imports - PASSED")
print("✅ Section 2: Configuration loaded - PASSED")
print("✅ Section 3: Private key validated - PASSED" if api_creds_ready else "⚠️ Section 3: Private key - SKIPPED (need private key)")
print("✅ Section 4: Client initialized - PASSED" if client else "⚠️ Section 4: Client - SKIPPED")
print("✅ Section 5: Markets fetched - PASSED" if len(markets) > 0 else "⚠️ Section 5: Markets - SKIPPED")
print("✅ Section 6: Market selected - PASSED" if token_ids else "⚠️ Section 6: Market selection - SKIPPED")
print("✅ Section 7: Order books fetched - PASSED" if orderbook_data else "⚠️ Section 7: Order books - SKIPPED")
print("✅ Section 8: Arbitrage calculated - PASSED" if orderbook_data else "⚠️ Section 8: Arbitrage - SKIPPED")
print("✅ Section 9: Fee calculator created - PASSED")
print("")
print("📋 What We've Learned:")
print("   ✅ How to connect to Polymarket CLOB")
print("   ✅ How to fetch market data")
print("   ✅ How to extract token IDs from markets")
print("   ✅ How to analyze order books")
print("   ✅ How to calculate arbitrage opportunities")
print("   ✅ How to estimate costs and profitability")
print("")
print("📋 Next Steps:")
print("   1. ✅ Private key configured")
print("   2. Fund wallet with USDC for live trading")
print("   3. Build WebSocket streaming for real-time data")
print("   4. Implement automated order placement")
print("   5. Add risk management and monitoring")
print("")
print("✅ Ready to build the production bot!")